# Задача 2.1. Аргументы



В данном задании Вам предлагается дописать код решения задачи бинарной классификации на основе классификатора KNN. Шаблон кода приведён Вам в качестве входных данных, Вам необходимо дополнить его так, чтобы алгоритм K ближайших соседей был настроен в следующей конфигурации:

- В качестве K должно быть выбрано значение 8
- В качестве метрики должно быть использовано манхэттенское расстояние

Для обучения модели разобьём выборку на тренировочную и тестовую при помощи функции train_test_split из модуля sklearn.model_selection. Подробнее об этой функции можно прочитать в [официальной документации sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html): 

Настройте разбиение на тестовую и тренировочную выборку таким образом, чтобы 0.3 объектов попали в тестовую выборку.

Оцените качество классификации на тестовой выборке при помощи [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) (функция подсчета доли правильных ответов из sklearn.metrics)

В качестве ответа укажите полученное качество на тестовой выборке, округлив ответ **вниз** до сотых.

In [1]:
import sklearn
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

random_seed = 4238
np.random.seed(random_seed)

X, y = load_breast_cancer(return_X_y=True)
X_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=True, random_state=42
)

clf = KNeighborsClassifier(n_neighbors=8, p=1)
clf.fit(X_train, y_train)

predictions = clf.predict(x_test)
acc = accuracy_score(y_test, predictions)

print(acc)

0.9766081871345029


## Примечания

1. **Не меняйте значение random_state и random_seed!** Это может привести к неправильным ответам.

2. Ответ должен быть записан в виде десятичной дроби через **точку** с двумя значащими цифрами. *Например: 0.11*

# Оптимальная метрика



В этом задании Вам предлагается подобрать оптимальную метрику и оптимальное значение гиперпараметра K из диапазона [1,50] для решения задачи классификации на примере датасета Ирисов Фишера. Этот датасет можно загрузить из модуля sklearn.datasets.

Качества оценивается при помощи метрики accuracy при помощи методики кросс-валидации. Об этой методике можно подробнее прочитать в [документации sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html).

Мы предлагаем Вам заполнить недостающие команды в следующем скелете кода и разобраться, какую метрику оптимально применять для решения данной задачи. В ответе на задание необходимо указать эту метрику.

Попробуйте 3 варианта: манхэттенское расстояние, евклидово расстояние и косинусное расстояние. Полный список возможных метрик можно посмотреть по [ссылке](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics). Меняйте этот параметр, изменяя значение аргумента `metric` при создании объекта класса `KNeighborsClassifier`. Найдите пару "метрика"-"K", для которой получается наилучшее качество и в качестве ответа укажите **найденную метрику**

**Замечание**: параметр *n_splits* - это количество разбиений `cv` в кросс-валидации. В качестве итоговой метрики берётся усреднение полученных значений метрик по всем разбиениям.

In [2]:
import sklearn
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

random_seed = 4238

np.random.seed(random_seed)
n_splits = 3

from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)

"""
  Здесь Вам предлагается написать тело цикла для подбора оптимального K
  Результаты оценки алгоритма при каждом отдельно взятом K рекомендуем записывать в список cv_scores
"""

metrics = ["manhattan", "euclidean", "cosine"]
cv_scores = []
best_score  = -1000
best_k      = -1
best_metric = None

for metric in metrics:
  for k in range(1, 51):
      
      clf = KNeighborsClassifier(n_neighbors=k, metric=metric)

      scores = cross_val_score(clf, X, y, cv=n_splits)
      mean_score = np.mean(scores)
      cv_scores.append(mean_score)

      if mean_score > best_score:
            best_score = mean_score
            best_k = k
            best_metric = metric
      # clf.fit(X_train, y_train)
      # predictions = clf.predict(x_test)
      # acc = accuracy_score(y, predictions)
      # cv_scores.append(acc)

print(f"\nBest metric: {best_metric}, Best k: {best_k}, Best score: {best_score}")


Best metric: euclidean, Best k: 4, Best score: 0.9866666666666667



В этом задании Вам предлагается написать класс `KNN_classifier`, пригодный для решения задачи классификации (многоклассовой).

Мы предлагаем Вам шаблон класса. В этом шаблоне заполните тела функций `.fit` и `.predict`

В качестве функции близости используйте Евклидово расстояние между объектами (подробнее https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html).

Напоминание:

* Функция `.fit(x, y)` производит обучение модели. В рамках этой функции необходимо реализовать подбор оптимальных параметров модели/сконфигурировать модель для дальнейшего использования на основе данной тренировочной выборки, где x - это матрица признакового описания выборки, а y - вектор ответов.

* Функция `.predict(x)` осуществляет предсказание для каждого из объектов, чьи векторные описания представлены строками матрицы x. Выполняется строго после `.fit()`. Ради безопасности можно даже реализовать механизм отказа в виде выбрасывания специальной ошибки `UnfittedError` в случае попытки вызова функции `.predict()` до вызова функции `.fit()`.

Замечание: не изменяйте названия класса и его методов. Это приведёт к ошибке при исполнении Вашего кода в процессе проверки задания. Тем не менее, Вы можете дописать свои собственные методы, если это необходимо.

Шаблон класса:

# Задача 2.3. KNN


In [3]:
import numpy as np

class KNN_classifier:
    def __init__(self, n_neighbors: int, **kwargs):
        self.K = n_neighbors
        self.X_train = None
        self.y_train = None

    def fit(self, x: np.array, y: np.array):
        """
        Сохраняет обучающие данные.
        
        Параметры:
        x (np.array): Массив признаков размерности n x m, где n - количество объектов, m - количество признаков.
        y (np.array): Массив меток размерности n.
        """
        self.X_train = np.array(x)
        self.y_train = np.array(y)

    def predict(self, x: np.array):
        """
        Предсказывает метки для входных данных.
        
        Параметры:
        x (np.array): Массив признаков размерности k x m, где k - количество объектов, m - количество признаков.
        
        Возвращает:
        np.array: Массив предсказанных меток размерности k.
        """
        if self.X_train is None or self.y_train is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit.")

        predictions = []
        for sample in x:
            # Вычисляем Евклидово расстояние между текущим объектом и всеми объектами обучающей выборки
            distances = np.linalg.norm(self.X_train - sample, axis=1)
            # Находим индексы K ближайших соседей
            nearest_indices = np.argsort(distances)[:self.K]
            # Получаем метки ближайших соседей
            nearest_labels = self.y_train[nearest_indices]
            # Определяем наиболее часто встречающуюся метку
            unique_labels, counts = np.unique(nearest_labels, return_counts=True)
            predicted_label = unique_labels[np.argmax(counts)]
            predictions.append(predicted_label)
        
        predictions = np.array(predictions)
        return predictions

## Примечания

1. Вы можете проверить правильность выполнения задания посредством сравнения полученных результатов с функцией из соответствующего модуля `sklearn`.

2. В рамках выполнения данного задания **запрещено** использовать функции из пакета `sklearn` и любого другого, кроме `numpy`. Код, использующий любые другие модули, не пройдёт тесты.

3. **Подсказка:** если Вы испытываете сложности с реализацией этого задания, начните выполнять его с написания функции `.predict`. В процессе написания этой функции Вы поймёте, что конкретно Вам требуется получить от обучающей выборки, какую информацию и в каком видед извлечь из неё. Затем реализуйте это в функции `.fit`

## Пример входных и выходных данных

In [4]:
X = [
    [ 0.56510722,  0.68599596, -0.92388505, -0.29546048, -0.12437532],
    [-0.79617537,  0.98406791,  1.19542652, -0.05626863, -0.69868076],
    [ 0.9629688 , -1.00423925, -0.53842833, -0.23744358,  0.83226685],
    [ 0.24671269, -0.41624448,  0.81679337,  1.59227446,  0.16192583],
    [-0.36972363,  0.17425997,  1.33668078,  1.16687907,  0.31709134],
    [-1.30482844, -0.05354323, -0.88862186, -1.121785  , -0.78442809],
    [-0.53975018,  0.90074877, -1.09317408,  1.52989481, -0.43375015],
    [-0.64709803, -0.09775791,  1.3506503 , -1.46957788,  1.63325543],
    [-0.73858464, -0.60678229,  0.31420272, -0.43100129, -0.37665876],
    [-0.29208809, -0.68795722,  0.06586655,  0.9583851 ,  1.70640775]
]

In [5]:
y = [1, 0, 0, 1, 0, 1, 0, 1, 0, 1]


In [6]:
knn = KNN_classifier(n_neighbors=3)


In [7]:
knn.fit(X, y)


In [8]:
x_test = [
    [-0.12489725,  0.65791923, -0.73112495,  1.42660225,  1.64728976],
    [ 0.01913388, -1.11351208, -0.63244098, -0.98121107,  0.38060892],
    [-0.92074931,  1.39812225,  0.39692147,  0.7717827 ,  0.44604002]
]

In [9]:
knn.predict(x_test)


array([1, 0, 0])